# Python notebook to show issues with Typing

## General Links

* OpenAI TypeSpec package: https://www.npmjs.com/package/@azure-tools/openai-typespec
* OpenAI TypeSpec repo: https://github.com/microsoft/openai-openapi-pr/tree/main/packages/openai-typespec/src
* Our current OpenAI TypeSpec files: https://github.com/Azure/azure-rest-api-specs-pr/tree/feature/ai-foundry/agents-v2/specification/ai/Azure.AI.Projects/.external-readonly/openai.external.typespec
* OpenAI Python reference: https://platform.openai.com/docs/api-reference 

In [ ]:
# Setup
from dotenv import load_dotenv
load_dotenv()


## Issues

### 'model' in ImageGenTool

OpenAI defines model as `Literal['gpt-image-1']` in our current TypeSpec, and `"gpt-image-1" | "gpt-image-1-mini"` in the [published OpenAI TypeSpec package](https://github.com/microsoft/openai-openapi-pr/blob/main/packages/openai-typespec/src/responses/models.tsp#L1742). But we try to pass in any str (with potentially different model name).

In [ ]:
import os
from typing import cast
from azure.ai.projects.models import ImageGenTool

image_generation_model = cast(str, os.getenv("IMAGE_GENERATION_MODEL_DEPLOYMENT_NAME"))

# OpenAI defines model as `Literal['gpt-image-1']` but we try to pass in any str (with potentially different mode name)
tool = ImageGenTool(model=image_generation_model, quality="low", size="1024x1024")

### '.evals.create'

Example taken from sample: samples\evaluations\sample_continuous_evaluation_rule.py
Link to OpenAI Python reference code for .evals.create(): https://platform.openai.com/docs/api-reference/evals/create

In [ ]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

endpoint = os.environ["AZURE_AI_PROJECT_ENDPOINT"]

with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=endpoint, credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
):

    data_source_config = {"type": "azure_ai_source", "scenario": "responses"}
    testing_criteria = [
        {"type": "azure_ai_evaluator", "name": "violence_detection", "evaluator_name": "builtin.violence"}
    ]
    eval_object = openai_client.evals.create(
        name="Continuous Evaluation",
        data_source_config=data_source_config,
        testing_criteria=testing_criteria,
    )
